In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@16d27cbf

In [2]:
import spark.implicits._
import java.sql.Date

case class Product(product_id: Int, new_price: Int, change_date: Date)

val product: Seq[(Int, Int, String)] = Seq(
  (1, 20, "2019-08-14"),
  (2, 50, "2019-08-14"),
  (1, 30, "2019-08-15"),
  (1, 35, "2019-08-16"),
  (2, 65, "2019-08-17"),
  (3, 20, "2019-08-18")
)

val productsDS = product.toDF("product_id", "new_price", "change_date")
  .withColumn("change_date", to_date(col("change_date")))
  .as[Product]

import spark.implicits._

import java.sql.Date


defined class Product
product: Seq[(Int, Int, String)] = List(
  (1, 20, "2019-08-14"),
  (2, 50, "2019-08-14"),
  (1, 30, "2019-08-15"),
  (1, 35, "2019-08-16"),
  (2, 65, "2019-08-17"),
  (3, 20, "2019-08-18")
)
productsDS: Dataset[Product] = [product_id: int, new_price: int ... 1 more field]

Write a solution to find the prices of all products on 2019-08-16. Assume the price of all products before any change is 10.

Return the result table in any order.

The result format is in the following example.

| product_id | price |
|------------|-------|
| 2          | 50    |
| 1          | 35    |
| 3          | 10    |

In [3]:
val productIdDF = productsDS.select(col("product_id"))
    .dropDuplicates()
    
val lastPrice = productsDS.filter(col("change_date") <= to_date(lit("2019-08-16")))
    .groupBy("product_id")
    .agg(max(col("change_date")).as("change_date"))
    .join(productsDS, Seq("product_id", "change_date"), "inner")

productIdDF.join(lastPrice, Seq("product_id"), "left_outer")
    .withColumn("price", when(col("new_price").isNull, 10).otherwise(col("new_price")))
    .select("product_id", "price")
    .show()

+----------+-----+
|product_id|price|
+----------+-----+
|         1|   35|
|         2|   50|
|         3|   10|
+----------+-----+



productIdDF: Dataset[Row] = [product_id: int]
lastPrice: DataFrame = [product_id: int, change_date: date ... 1 more field]

In [4]:
spark.stop()